In [162]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import cv2

# Read Depth Data

In [160]:
# For reading depth camera
def read_depth_camera(dcamera_path, show_video, nw_resize=1, nh_resize=1):
    video  = cv2.VideoCapture(dcamera_path)
    ret, frame = video.read()
    
    # Get total # of frame count 
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
    frame_height = int(frame.shape[0])
    frame_width = int(frame.shape[1])

    
    depth_frames = np.empty((frame_count, int(frame_height/nh_resize), int(frame_width/nw_resize)))
    depth_frames = np.empty((frame_count, int(frame_height/nh_resize), int(frame_width/nw_resize),3))
    count = 0
    while (video.isOpened()):
        ret, frame = video.read()
        
        if ret == True:
            gray_frame = frame
            gray_frame = cv2.resize(gray_frame, \
                                    (int(frame_width/nw_resize), int(frame_height/nh_resize)),\
                                    interpolation = cv2.INTER_NEAREST)

            depth_frames[count] = gray_frame
            if show_video == True:
                cv2.imshow("Depth", gray_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            count = count + 1
        else: 
            break
            

    video.release()
    #cv2.destroyAllWindows()
    return depth_frames


In [163]:
n_test = (24,30,31,32,33,35)
nw_resize = 2 # for reducing width
nh_resize = 2 # for reducing height
xtemp = {}
show_video = 0
subj = ['leo','leo','leo','leo','leo','leo']
        
for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
    data_dir = os.path.join(r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    train_dcamera_path = os.path.join(data_dir , 'depth_processed_'+subj[i]+'_test'+str(n_test[i])+'.avi')
    xtemp[i] = read_depth_camera(train_dcamera_path, show_video, nw_resize=nw_resize, nh_resize=nh_resize).astype('uint8')
    


In [164]:
tlen=0 # total length of training data set
for x in range(len(xtemp)):
    tlen+= xtemp[x].shape[0]


x_train = np.zeros((tlen,xtemp[0].shape[1],xtemp[0].shape[2],xtemp[0].shape[3]),dtype='uint8') # initialize training set data
xrun_cum = 0
for i in range (len(xtemp)):
    xrun_n = len(xtemp[i])
    x_train[xrun_cum:xrun_cum+xrun_n,:,:,:] = xtemp[i][:xrun_n,:,:,:] # compiling all the training data into one large array
    xrun_cum += xrun_n

# Read Force Data

In [165]:
n_test = (24,30,31,32,33,35)
date = ('11_15_2020','11_24_2020','11_24_2020','11_25_2020','11_25_2020','11_25_2020')
subj = ['leo','leo','leo','leo','leo','leo']
subjwgt = [67, 67, 67, 67, 67, 67]
subjht = [174, 174, 174, 174, 174, 174]
xfcss_gt = {}
yrun = 0

for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
    data_dir = os.path.join(r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    fcss_data_dir = os.path.join(data_dir , 'fcss_processed_'+subj[i]+'_' + test_str + '_' + date[i] + '.txt')
    xfcss_gttemp = pd.read_csv(fcss_data_dir)/subjwgt[i]
    xfcss_gt[i]=xfcss_gttemp
    if i == 0:
        xfcss_train = xfcss_gttemp
    else:
        xfcss_train = pd.concat([xfcss_train,xfcss_gt[i]],axis=0)
del xfcss_gt

In [149]:
def compute_moment(fdss_data):
    d1 = 0.19
    d2 = 0.08
    d3 = 0.19
    g = 9.81
    
    N = len(fdss_data)
    moment = np.zeros((N,3))
    
    for i in range(N):
        moment[i,0] = ((fdss_data[i, 1] - fdss_data[i, 2]) * d3  + fdss_data[i,5] * d2)* g 
        
        moment[i,1] = ((fdss_data[i, 1] + fdss_data[i, 2] - fdss_data[i,0]) * d1 + (fdss_data[i, 3] + fdss_data[i, 4]) * d2) * g
        
        moment[i,2] = (fdss_data[i,3] - fdss_data[i,4]) * d3 * g


    return moment

In [150]:
moment = compute_moment(xfcss_train.values)

# Read Output Data

In [3]:
def read_output_data(qtm_file_data, theta):
    if theta=='x':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Lean Left/Right Angle (deg)"])
    if theta=='y':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Lean Forward/Backwards Angle (deg)"])
    if theta=='z':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Torso Twist Angle (deg)"])
        
    
    return qtm_data

In [4]:
n_test = (24,30,31,32,33,35)
date = ('11_15_2020','11_24_2020','11_24_2020','11_25_2020','11_25_2020','11_25_2020')
subj = ['leo','leo','leo','leo','leo','leo']
y_gt = {}
yrun = 0
theta_interest = 'z'
for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
    data_dir = os.path.join(r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    qtm_file_data_dir = os.path.join(data_dir , 'qtm_processed_'+subj[i]+'_test' + str(n_test[i]) + '_' + date[i] + '.txt')
    y_gt[i] = read_output_data(qtm_file_data_dir,theta_interest).values

    

In [5]:
tlen=0
for x in range(len(y_gt)):
    tlen+= y_gt[x].shape[0]
    
yrun_cum = 0
y_train = np.zeros((tlen,1))
for i in range (len(y_gt)):
    yrun_n = len(y_gt[i])
    y_train[yrun_cum:yrun_cum+yrun_n] = y_gt[i][:]
    yrun_cum += yrun_n

# Divide into training, validation, and testing dataset

In [187]:
nsamps = xfcss_train.shape[0]
n60p = int(np.floor(nsamps*0.6))
n80p = int(np.floor(nsamps*0.8))

X = x_train[0:n60p]/255.
TrainX2 = xfcss_train.values[0:n60p]
TrainY = y_train[0:n60p]

X_val = x_train[n60p:n80p]/255.
ValX2 = xfcss_train.values[n60p:n80p]
ValY = y_train[n60p:n80p]

X_test = x_train[n80p:nsamps]/255.
TestX2 = xfcss_train.values[n80p:nsamps]
TestY = y_train[n80p:nsamps]


In [172]:
nsamps = moment.shape[0]
n60p = int(np.floor(nsamps*0.6))
n80p = int(np.floor(nsamps*0.8))

TrainX = moment[0:n60p]
TrainY = y_train[0:n60p]

ValX = moment[n60p:n80p]
ValY = y_train[n60p:n80p]

TestX = moment[n80p:nsamps]
TestY = y_train[n80p:nsamps]


# Normalize X data

In [180]:
max_x = np.zeros((6,1))
X2 = np.zeros(TrainX2.shape)
X2_val = np.zeros(ValX2.shape)
X2_test = np.zeros(TestX2.shape)
for i in range(6):
    max_x[i] = max(abs(TrainX2[:,i]))
    X2[:,i] = TrainX2[:,i]/max_x[i]
    X2_val[:,i] = ValX2[:,i]/max_x[i]
    X2_test[:,i] = TestX2[:,i]/max_x[i]
    
plt.plot(X2_test)

# Discretize Output Data 

In [181]:
n_bin = 3
min_val = -50
max_val = 50

y_bins = np.linspace(min_val, max_val, n_bin)
print(y_bins)
Y_raw = np.digitize(y, y_bins) - 1
Y_val_raw = np.digitize(y_val, y_bins) - 1
Y_test_raw = np.digitize(y_test, y_bins) -1 

Y = tf.keras.utils.to_categorical(Y_raw, num_classes=n_bin-1, dtype='float32')
Y_val = tf.keras.utils.to_categorical(Y_val_raw, num_classes=n_bin-1, dtype='float32')
Y_test = tf.keras.utils.to_categorical(Y_test_raw, num_classes=n_bin-1, dtype='float32')


plt.figure(figsize=(15,5))
plt.plot(Y_val_raw)


[-50.   0.  50.]


# Create model 

In [198]:
# Create Neural Netowrk

from tensorflow.keras.layers import Bidirectional, Conv2D, MaxPooling2D, Input, concatenate, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Dropout, Reshape, Permute, Flatten
from tensorflow.keras.models import Model
dropout_rate = 0.2

model_start = Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
model_start2 = Input(shape=(xfcss_train.shape[1],))
model_cnn = model_start
model_perc = model_start2

model_cnn = Conv2D(filters=8, kernel_size=(3, 3),padding='same')(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_perc = Dense(32)(model_perc)
model_perc = Activation('relu')(model_perc)

model_cnn = Conv2D(filters=16, kernel_size=(3, 3),padding='same')(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)


model_cnn = Conv2D(filters=32, kernel_size=(3, 3),padding='same')(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_cnn = Conv2D(filters=64, kernel_size=(3, 3),padding='same')(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_cnn = Flatten()(model_cnn)


model_comb = concatenate([model_cnn,model_perc],axis=-1)

model_comb = Dense(256)(model_comb)
model_comb = Activation('relu')(model_comb)
model_comb = Dropout(dropout_rate)(model_comb)

output = Dense(n_bin-1)(model_comb)
output = Activation('softmax', name='probability')(output)
model = Model(inputs=[model_start,model_start2],outputs=output)

model.compile(
    optimizer = 'adam',
#     loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy'])


# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True) #Moving to 1000 patience. 
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50,restore_best_weights=True) #Moving to 1000 patience. 

In [196]:
# Create Neural Netowrk
from tensorflow.keras import layers

from tensorflow.keras.layers import Bidirectional, Conv2D, MaxPooling2D, Input, concatenate, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Dropout, Reshape, Permute, Flatten
from tensorflow.keras.models import Model




model = tf.keras.Sequential([
    layers.InputLayer(input_shape = (X.shape[1],)),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(n_bin-1)
])


model.compile(
    optimizer = 'adam',
#     loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#     loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ['accuracy'])

model.summary()

# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50,restore_best_weights=True) #Moving to 1000 patience. 

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 256)               15616     
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_62 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_64 (Dense)             (None, 2)               

In [195]:
Y.shape

(49428, 2)

In [199]:
a1 = time.perf_counter() 
history = model.fit([X, X2], Y, batch_size = 128, epochs = 10, callback = [callback],
                    validation_data = ([X_val, X2_val], Y_val),verbose=1)
a2 = time.perf_counter() 


Epoch 1/10


C:\Users\77bis\anaconda3\envs\python3.6\lib\site-packages\tensorflow\python\keras\backend.py:5095: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


387/387 [==============================] - 65s 167ms/step - loss: 0.6077 - accuracy: 0.6640 - val_loss: 0.5826 - val_accuracy: 0.6918
Epoch 2/10
387/387 [==============================] - 58s 149ms/step - loss: 0.4361 - accuracy: 0.8047 - val_loss: 0.5414 - val_accuracy: 0.7332
Epoch 3/10
387/387 [==============================] - 59s 152ms/step - loss: 0.3909 - accuracy: 0.8316 - val_loss: 0.5536 - val_accuracy: 0.7346
Epoch 4/10
387/387 [==============================] - 59s 153ms/step - loss: 0.3684 - accuracy: 0.8394 - val_loss: 0.5557 - val_accuracy: 0.7279
Epoch 5/10
387/387 [==============================] - 58s 151ms/step - loss: 0.3438 - accuracy: 0.8515 - val_loss: 0.5918 - val_accuracy: 0.7302
Epoch 6/10
387/387 [==============================] - 59s 153ms/step - loss: 0.3219 - accuracy: 0.8609 - val_loss: 0.6193 - val_accuracy: 0.7179
Epoch 7/10
387/387 [==============================] - 58s 150ms/step - loss: 0.3004 - accuracy: 0.8702 - val_loss: 0.6156 - val_accuracy: 0.6

In [200]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


In [201]:
def differences(a, b):
    if len(a) != len(b):
        raise ValueError("Lists of different length.")
    return sum(i != j for i, j in zip(a, b))

In [203]:
y_pred_prob = model.predict([X_test, X2_test])
y_pred = np.argmax(y_pred_prob, axis = 1)
plt.figure(figsize=(20,6))
plt.plot(Y_test_raw,'k')
plt.plot(y_pred,'r--')
plt.title('Prediction of Training Set (Sanity Check)')
plt.legend(labels=['Ground Truth','Prediction'])
plt.show()

diff = differences(y_pred, Y_test_raw)
print(diff/len(Y_test_raw) * 100)

[35.29768769]
